In [1]:
from indra.tools.assemble_corpus import filter_grounded_only
from indra.statements.statements import stmts_from_json
from indra.statements.statements import stmts_to_json
from indra_db.client.principal import raw_statements
from indra.tools import assemble_corpus as ac
from collections import defaultdict
from indra_db.util import get_db
from collections import Counter
from datetime import datetime
import pickle as pk
import pandas as pd
import logging
import gilda
import time
db = get_db('primary')
log = logging.getLogger('Pain')

In [ ]:
# Read the tcids
with open('./id_list.txt', 'r') as fh:
    tcid = [i.strip() for i in fh]
stmts = raw_statements.get_raw_stmt_jsons(clauses=[db.TextContent.id.in_(tcid)], db=db)

# Dump the raw stmts
stmts_pkl = open('./raw_stmts_json.pkl', 'wb')
pk.dump(stmts, stmts_pkl)

# Convert stmts from json to list
stmts_json = list(stmts.values())
stmts_list = stmts_from_json(stmts_json)
# dump the stmts list
outfh = open('./raw_stmts_list.pkl', 'wb')
pk.dump(stmts_list, outfh)



In [2]:
# Read the stmts list and ground the agents
infh = open('./raw_stmts_list.pkl', 'rb')
stmts_list = pk.load(infh)

start_time = time.time()
grounded_stmts = ac.map_grounding(stmts_list)
end_time = time.time()
round((end_time - start_time)/60, 2)

'''
INFO: [2021-10-06 11:27:28] indra.tools.assemble_corpus - Mapping grounding on 1333769 statements...
INFO: [2021-10-06 11:35:01] indra.preassembler.grounding_mapper.mapper - 44784 statements filtered out
7.56
'''

outfh = open('./grounded_stmts.pkl', 'wb')
pk.dump(grounded_stmts, outfh)

filtered_stmts = filter_grounded_only(grounded_stmts)
fh = open('./filter_grounded_only_stmts.pkl', 'wb')
pk.dump(filtered_stmts, fh)

INFO: [2021-10-14 09:54:50] indra.tools.assemble_corpus - Mapping grounding on 1333769 statements...
INFO: [2021-10-14 09:54:51] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/sbunga/.indra/bio_ontology/1.12/bio_ontology.pkl
INFO: [2021-10-14 10:41:56] indra.preassembler.grounding_mapper.mapper - 44784 statements filtered out


In [2]:
with open('../data/filter_grounded_only_stmts.pkl', 'rb') as fh:
    filtered_stmts = pk.load(fh)

In [3]:
filtered_stmts

[Activation(env(), Receptors, Opioid, kappa()),
 Dephosphorylation(TGFB(), Cofilin()),
 Inhibition(quercetin(), Collagen()),
 Inhibition(quercetin(), TGFB()),
 Inhibition(quercetin(), inflammatory response()),
 Inhibition(quercetin(), TGFB()),
 Inhibition(quercetin(), TGFB()),
 Inhibition(quercetin(), Cofilin()),
 Inhibition(quercetin(), DNA Damage()),
 Activation(TGFB(), cell differentiation()),
 Activation(quercetin(), signal transduction()),
 Activation(quercetin(), TGFB()),
 Inhibition(quercetin(), signal transduction()),
 Activation(branch(), back()),
 Inhibition(etoricoxib(), morphine(1+)()),
 Inhibition(etoricoxib(), morphine(1+)()),
 Inhibition(etoricoxib(), paracetamol()),
 Inhibition(etoricoxib(), hydrocodone()),
 Activation(SERPINA5(), str()),
 Activation(serotonin(), PRL()),
 Activation(CNR2(), inflammatory response()),
 Activation(Th1 Cells(), IFNG()),
 Activation(Th1 Cells(), TBX21()),
 Inhibition(chromogranin(), SLPI()),
 Inhibition(chromogranin(), bilirubin IXalpha()),


In [21]:

groundings = []
for stmt in grounded_stmts:
    for ag in stmt.real_agent_list():
        db_ns, db_id = ag.get_grounding()
        if db_ns and db_id:
            gr_ag, gr_ns, gr_id = ag.name, db_ns, db_id
        else:
            for k, v in ag.db_refs.items():
                if k not in {'TEXT', 'TEXT_NORM'}:
                    gr_ag, gr_ns, gr_id = ag.name, k, v
                    break
            else:
                gr_ag, gr_ns, gr_id = ag.name, '', ''
                

        try:
            groundings.append((gr_ag, ag.db_refs['TEXT'], gr_ns, gr_id))
        except KeyError:
            pass

In [52]:

cnt = Counter(groundings)
df = []
for (gr_ag, txt, gr_ns, gr_id), count in cnt.most_common():
    df.append(
    {
        'ag_name': gr_ag,
        'text': txt,
        'db_ns': gr_ns,
        'db_id': gr_id,
        'count': count
    })
grounded_df = pd.DataFrame(df)
grounded_df = grounded_df[grounded_df['count'] > 1]
grounded_df.to_csv('./agent_groundings.csv', sep=',', index=False)

In [43]:
#pd.DataFrame(df)
for (txt, gr_ns, gr_id), c in cnt.most_common():
    print(txt, gr_ns, gr_id, c)
    break

pain MESH D010146 81133


In [10]:
er = [{"type": "Phosphorylation", "sub": {"name": "x", "db_refs": {"TEXT": "ER"}}, "belief": 1, "evidence": [{"pmid": "26303083", "text_refs": {"PMID": "26303083"}, "source_hash": -3994646362831840617}], "id": "cbb6c8dd-2e53-42ba-b6e5-e7063ec3f297", "matches_hash": "-22906490441916092"}]
er = stmts_from_json(er)
er = ac.map_grounding(er)
er

INFO: [2021-10-07 16:34:38] indra.tools.assemble_corpus - Mapping grounding on 1 statements...
INFO: [2021-10-07 16:34:38] indra.preassembler.grounding_mapper.mapper - 0 statements filtered out


[Phosphorylation(None, ESR())]